## 0.0. Imports

In [36]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import  mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import seaborn as sns
import plotly.express as px
import sweetviz as sv
import pickle
from boruta import BorutaPy
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor



pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
df_raw = pd.read_csv('../data/treino.csv')
df_test = pd.read_csv('../data/teste.csv')

### 1.0. Data Description

df1 = df_test.copy()

# rename columns

df1.columns = ['id', 'num_fotos', 'marca', 'modelo', 'versao', 'ano_de_fabricacao',
       'ano_modelo', 'odometro', 'cambio', 'num_portas', 'tipo', 'blindado',
       'cor', 'tipo_vendedor', 'cidade_vendedor', 'estado_vendedor',
       'tipo_anuncio', 'entrega_delivery', 'troca', 'elegivel_revisao',
       'aceita_troca', 'dono_unico',
       'todas_revisoes_concessionaria',
       'ipva_pago', 'licenciado',
       'garantia_de_fabrica',
       'todas_revisoes_agenda',
       'alienado']

#fill na

df1['num_fotos'] = df1['num_fotos'].fillna(0)

df1 = df1.drop('alienado', axis=1)

na_cols = df1.columns[-8:]

for col in na_cols:
    df1[col] = np.where(df1[col].isna(), 0, 1)

df1[df1.T.tail(8).index] = df1[df1.T.tail(8).index].astype('int64')

df1['ano_modelo'] = df1['ano_modelo'].astype('int64')
df1['num_fotos'] = df1['num_fotos'].astype('int64')

## 2.0. Data Filtering

import sweetviz as sv
# my_report = sv.analyze(df1, target_feat='preco')
# my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

df1 =df1.drop(['elegivel_revisao'], axis=1)

### 3.1. Rescaling

min_max_cols = [
 'ano_de_fabricacao',   
 'ano_modelo',          
 'odometro',            
 'num_portas']


for column in min_max_cols:
    mms = pickle.load( open(f'../parameters/{column}_scaler.pkl', 'rb'))
    df1[column] = mms.fit_transform(df1[[column]].values)
    

# df1['preco'] = np.log1p(df1['preco'])

### 3.2. Encoding

df1 = df1.drop(['cidade_vendedor', 'tipo_anuncio'], axis=1)

#get uf
df1['estado_vendedor'] = df1['estado_vendedor'].apply(lambda x: x[-3:-1])

map_cor = {'Preto':'preto', 'Branco':'branco', 'Prata':'prata', 'Cinza':'cinza', 'Dourado':'outros', 'Vermelho':'outros', 'Azul':'outros',
       'Verde':'outros'}

# map_regiao={'SP':'sudeste','RS':'sul','MG':'sudeste','PR':'sul','RJ':'sudeste','MA':'nordeste','SC':'sul','AL':'nordeste','BA':'nordeste','GO':'centro_oeste','RN':'nordeste','PE':'nordeste','MT':'centro_oeste','PA':'norte','CE':'nordeste','AM':'nordeste','ES':'sudeste','RO':'norte','PB':'nordeste','TO':'norte','AC':'norte','SE':'nordeste','MS':'centro_oeste','RR':'norte','PI':'nordeste'}


map_cambio = {'Automática': 2, 'Manual' :0, 'CVT' :2, 'Automatizada': 2, 'Semi-automática': 1,
       'Automatizada DCT' : 2, 'Automática Sequencial' : 2}

In [37]:
# target encoders
for column in ['marca', 'modelo', 'versao', 'tipo', 'estado_vendedor']:
    print(column)
    target = pickle.load( open(f'../parameters/{column}_encode.pkl', 'rb'))
    df1[column] = df1[column].map(target)

marca
modelo
versao
tipo
estado_vendedor


In [38]:
# binary
df1['blindado'] = np.where(df1['blindado']=='N', 0, 1)

#map
df1['cor'] = df1['cor'].map(map_cor)
# df1['estado_vendedor'] = df1['estado_vendedor'].map(map_regiao)
df1['cambio'] = df1['cambio'].map(map_cambio)

# one hot encoding
df1 = pd.get_dummies(df1, columns=['cor',   'tipo_vendedor'])


for column in df1.select_dtypes(['int32', 'uint8']).columns:
    df1[column] = df1[column].astype('int64')

## 4.0. Feature Selection

df2 = df1.drop('id', axis=1).copy()

In [43]:
cols_selected = ['versao', 'modelo', 'cambio', 'odometro', 'tipo', 'ano_modelo',
       'marca', 'ano_de_fabricacao']

X = df2[cols_selected]
## Model Train


#model definition


model = pd.read_pickle('../parameters/model.pkl')

In [44]:
model

LGBMRegressor(learning_rate=0.07, max_depth=80, num_leaves=90)

In [45]:
cols_selected

['versao',
 'modelo',
 'cambio',
 'odometro',
 'tipo',
 'ano_modelo',
 'marca',
 'ano_de_fabricacao']

In [46]:
#model predict
y_hat = model.predict(X[cols_selected])

# y_test_ = np.expm1(y_test)
y_hat_ = np.expm1(y_hat)



# y_test_ = np.expm1(y_test)

df_final = pd.DataFrame()
df_final['ID'] = df_test['ID']
df_final['preco'] = y_hat_

df_final.to_csv('../data/sample.csv', index=False)

In [47]:
df_final

,ID,preco
0,24813264385557040124808779273028388499,69654.708791
1,295636316453795508942188530111300065666,93943.149757
2,101258309166227950735244624080888109884,81049.899021
3,28348734455782469411126661985772047409,82969.200846
4,193163160502972147671913739170248305797,91453.725342
...,...,...
39441,238233399351588823822117090805568390727,35431.035839
39442,64621912306231118962468441892654163025,108247.323466
39443,100311033226508317456901122129284293382,123405.350716
39444,217317181330151694133399005110777689124,374934.347779


In [ ]:
df_final